In [ ]:
from selenium import webdriver;
from bs4 import BeautifulSoup;
from selenium.webdriver.chrome.service import Service;
from selenium.webdriver.common.by import By;
from selenium.webdriver.chrome.options import Options;
from selenium.webdriver.common.action_chains import ActionChains;
from webdriver_manager.chrome import ChromeDriverManager;
import time;
import pandas as pd;
import re;


In [4]:

# *open window

options  = Options();
options.add_experimental_option("detach", True);
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options);

driver.get('https://www.lamudi.com.ph/apartment/rent/'); # *link to open
driver.maximize_window(); # *maximize the window

# * data to get : 
# *title, location, price, key-info : bedroom, bath, floor area, agent


# ! step 1 : get max page and curr page


soup = BeautifulSoup(driver.page_source, "html.parser");
page = driver.find_element(By.CLASS_NAME, "nav-box-center").find_element(By.TAG_NAME ,'select');

curr_page = int(page.get_attribute('data-pagination-current'));
max_page = int(page.get_attribute('data-pagination-end'));

print(curr_page, max_page)
rows = [] 

# cnt = 0
while curr_page <= max_page:
    soup = BeautifulSoup(driver.page_source, "html.parser");
    page = driver.find_element(By.CLASS_NAME, "nav-box-center").find_element(By.TAG_NAME ,'select');
    for item in soup.find_all('div', class_ = 'ListingCell-wrapper'): # selecting the card wrapper
        #getting listing details : title,loc,desc,price,feaures/ammenities,agent,agent_details
        title = item.find('h2',"ListingCell-KeyInfo-title").text.strip(); 
        location = item.find('span',"ListingCell-KeyInfo-address-text").text.strip();
        desc = item.find('div',"ListingCell-shortDescription").text.strip();
        price = item.select_one(".ListingCell-KeyInfo-price > :first-child").text.strip();
        features_holder = item.select(".ListingCell-keyInfo-details > .KeyInformation_v2 > .KeyInformation-attribute_v2");
        z = [];
        for x in features_holder: # looping through feautures
            # print(x)
            ammenity =  x.find('span', 'KeyInformation-label_v2').text.strip();
            amount = x.select_one("div > span").text.strip();
            y = ammenity + " : " + amount;
            print(y)
            z.append(y);
        features = z 

        agent = item.find('div',"ListingDetail-agent-name").text.strip();
        membership = item.find('div',"ListingDetail-agent-membership").text.strip();
        verified = item.find('img',"ListingDetail-agent-verified-icon");
        
        if(str(verified) != 'None'): # setting agent verified status
            verified = 'verified'
        else:
            verified = 'not verified'
        # print (verified)
        print('title' ,title)
        rows.append([title,location,desc,price,features,agent,membership,verified]) # appending to rows

    time.sleep(2);
    actions = ActionChains(driver);

    toAction = driver.find_element(By.CLASS_NAME,"LuxWidget-Title");
    actions.move_to_element(toAction).perform();

    time.sleep(2);
    actions = ActionChains(driver);
    try:
        nextbtn = driver.find_element(By.CSS_SELECTOR,".next > a");
        actions.click(nextbtn); 
        actions.perform();
    except:
        print('no button found')

    curr_page = curr_page + 1;
    time.sleep(5);


   

Bedroom : 1
Bath : 1
Floor area : 60 m²
title 1-BR Apartment for Rent at Ferros Bel Air Tower, Makati City
Bedrooms : 6
Baths : 6
Floor area : 320 m²
title Staff House or Office Space near CIRCUIT Mall Whole Building
Bedrooms : 2
Baths : 2
title 2 bedroom apartment for rent in executive subdivision in antipolo!!!
Bedroom : 1
Bath : 1
Floor area : 20 m²
title Studio Room for Rent in Tejeros, Makati Convenient Location
Bedroom : 1
Bath : 1
Floor area : 36 m²
title Safestay Apartment for rent at Mactan Airport, Lapu-Lapu, Cebu
Bedrooms : 12
Baths : 6
Floor area : 256 m²
title 12 Bedroom Condo for Rent in Makati, Metro Manila
Bedrooms : 9
Baths : 4
Floor area : 150 m²
title 9 Bedroom Condo for Rent in Bangkal, Metro Manila in Bangkal, Makati
Bedroom : 1
Bath : 1
Floor area : 25 m²
title Php 7,500/month across University of San Carlos with window private CR, Cebu
Bedrooms : 3
Baths : 3
Floor area : 217 m²
title Apartment for RENT Upper Cubi, Subic Bay Freeport Zone
Bedroom : 1
Bath : 1
Floo

KeyboardInterrupt: 

In [ ]:
# rows
df = pd.DataFrame(rows,
                  columns=['title','location','description','price','features','agent','agent_membership','agent_verified'])

df

In [ ]:
import datetime

today_d =  datetime.datetime.now().strftime('%m-%d-%y')
file_name = 'lamudi-' + today_d + '.csv';
file_name


In [ ]:
from pathlib import Path  
filepath = Path(r'C:\Users\Jake\Desktop\YEJI\\3Y2S\Analytics Techniques and Tools\WEB_CRAWLING\\appartment_rental\\'+file_name)  
filepath.parent.mkdir(parents=True, exist_ok=True)  
df.to_csv(filepath)
